In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np


# 2. Load dataset (use your processed/train split)
df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered.csv")

pd.set_option('display.max_columns', None)  # Show all columns in the output
pd.set_option('display.max_rows', None)     # Show all rows in the output

In [2]:
print(df.shape)
df.head(2)

(795682, 41)


,year,quarter,month,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Total Population,Median Age,Per Capita Income,Total Families Below Poverty,Total Housing Units,Median Rent,Median Home Value,Total Labor Force,Unemployed Population,Total School Age Population,Total School Enrollment,Median Commute Time,price,lat,lng,zipcode_freq,city_encoded
0,2014,4,12,159900.0,98700.0,114.468864,66.498930,5.0,0.0,6.0,18.0,209.0,0.928091,0.000000,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,3187.0,41.4,22328.0,3187.0,1300.0,825.0,142800.0,1612.0,385.0,3101.0,3101.0,1182.0,144196.333365,38.7352,-90.3501,123,172651.841291
1,2016,3,9,215000.0,222900.0,114.774973,116.016713,110.0,105.0,111.0,81.0,44.0,0.990445,0.245455,0.428571,2.0,1.0,0.0,0.0,4.0,13.0,20.0,0.0,5.0,19968.0,32.8,24060.0,19960.0,7502.0,1347.0,161200.0,10726.0,687.0,18678.0,18678.0,9332.0,208628.100288,28.4343,-81.3630,125,244403.845857


In [3]:
high_vif_features = [
    "Total School Enrollment",
    "Total School Age Population",
    "Total Population",
    "Total Families Below Poverty",
    "Total Labor Force",
    "Median Commute Time"
]

df.drop(columns=high_vif_features, inplace=True)

In [4]:
print(df.shape)
df.head(2)

(795682, 35)


,year,quarter,month,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,pending_sales,new_listings,inventory,median_dom,avg_sale_to_list,sold_above_list,off_market_in_two_weeks,bank,bus,hospital,mall,park,restaurant,school,station,supermarket,Median Age,Per Capita Income,Total Housing Units,Median Rent,Median Home Value,Unemployed Population,price,lat,lng,zipcode_freq,city_encoded
0,2014,4,12,159900.0,98700.0,114.468864,66.498930,5.0,0.0,6.0,18.0,209.0,0.928091,0.000000,0.200000,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,41.4,22328.0,1300.0,825.0,142800.0,385.0,144196.333365,38.7352,-90.3501,123,172651.841291
1,2016,3,9,215000.0,222900.0,114.774973,116.016713,110.0,105.0,111.0,81.0,44.0,0.990445,0.245455,0.428571,2.0,1.0,0.0,0.0,4.0,13.0,20.0,0.0,5.0,32.8,24060.0,7502.0,1347.0,161200.0,687.0,208628.100288,28.4343,-81.3630,125,244403.845857


In [5]:
# Define target and features
target = "price"   # <-- adjust if needed
X = df.drop(columns=[target])
y = df[target]

# Train/validation split (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)

Train shape: (636545, 34)
Validation shape: (159137, 34)


In [6]:
# ================================================
# Standardization (Z-score scaling)
# ================================================
scaler = StandardScaler()

# Fit on training data, transform both train & val
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print("Scaled data shape:", X_train_scaled.shape)

Scaled data shape: (636545, 34)


In [7]:
X_train_scaled

array([[-1.04798489, -1.37007196, -1.62822559, ..., -1.26847371,
         0.88133958, -0.36606214],
       [-1.04798489,  1.32965456,  1.00064848, ...,  0.5067477 ,
         2.30290438, -0.73907026],
       [-1.63404   , -0.47016312, -0.16774   , ..., -0.83903765,
         1.4499655 ,  0.04412302],
       ...,
       [-0.75495733,  1.32965456,  1.00064848, ...,  0.60306758,
         0.88133958, -0.82685502],
       [-0.16890222,  0.42974572,  0.41645424, ..., -1.87263316,
         0.0284007 ,  3.31115156],
       [-1.63404   ,  1.32965456,  1.58484271, ...,  0.65885124,
         0.88133958, -1.21752817]])

In [8]:
# ================================================
# Linear Regression (no regularization)
# ================================================
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

y_pred_lr = lr.predict(X_val_scaled)

print("Linear Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_lr))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_lr)))
print("R²:", r2_score(y_val, y_pred_lr))


Linear Regression performance:
MAE: 45486.928757838854
RMSE: 112216.35729122824
R²: 0.8941450558095324


- This difference leads to Lasso performing feature selection by potentially setting some coefficients to zero, 
- whereas Ridge regression shrinks all coefficients towards zero but doesn't eliminate any

In [9]:
# ================================================
# Ridge Regression (L2 regularization) --> Shrinks but doesn't eliminate features
# ================================================
ridge = Ridge(alpha=1.0)  # alpha controls regularization strength
ridge.fit(X_train_scaled, y_train)

y_pred_ridge = ridge.predict(X_val_scaled)

print("Ridge Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_ridge))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_ridge)))
print("R²:", r2_score(y_val, y_pred_ridge))


Ridge Regression performance:
MAE: 45486.89344326461
RMSE: 112216.47927773595
R²: 0.8941448256669348


In [10]:
# ================================================
# Lasso Regression (L1 regularization). --> can eliminate some features
# ================================================
lasso = Lasso(alpha=0.1)  # alpha controls regularization strength
lasso.fit(X_train_scaled, y_train)

y_pred_lasso = lasso.predict(X_val_scaled)

print("Lasso Regression performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_lasso))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_lasso)))
print("R²:", r2_score(y_val, y_pred_lasso))

Lasso Regression performance:
MAE: 45486.78202897442
RMSE: 112216.33335463326
R²: 0.8941451009688428


In [11]:
# ================================================
# ElasticNet (L1 + L2 regularization) --> Combination of both
# ================================================
elastic = ElasticNet(alpha=0.1, l1_ratio=0.5)  
# l1_ratio=0.5 → equal mix of L1 and L2
elastic.fit(X_train_scaled, y_train)

y_pred_elastic = elastic.predict(X_val_scaled)

print("ElasticNet performance:")
print("MAE:", mean_absolute_error(y_val, y_pred_elastic))
print("RMSE:", np.sqrt(mean_squared_error(y_val, y_pred_elastic)))
print("R²:", r2_score(y_val, y_pred_elastic))


ElasticNet performance:
MAE: 45452.90248008058
RMSE: 116547.05270731628
R²: 0.8858170117166875


- Scaling worked → all models gave stable results.

- Linear Regression is a strong baseline → regularization didn’t add much.

- Collinearity exists but isn’t catastrophic → dropping features / ratios may help, but Ridge didn’t really need to fix it.

- Multicollinearity does not hurt predictive accuracy much → the model was already using redundant features that all point in the same direction.

- Dropping them removed redundancy, but the predictive signal remained, so performance didn’t collapse.